# A population simulator using fitness landscapes
With reference to [this paper](https://www.nature.com/articles/s41559-016-0007.pdf) and [its supplement](https://media.nature.com/original/nature-assets/natecolevol/2016/s41559-016-0007/extref/s41559-016-0007-s1.pdf).

In [1]:
# Preliminary stuff
using DataFrames, CSV, Plots, Distributions
plotlyjs()
IJulia.clear_output()

const probability_mutation = 1.0e-8
const carrying_capacity = 1.0e+9
;

Genotypes are represented as binary strings. 0 refers to a wild-type loci, 1 refers to a mutant. Since we know there are 4 loci, we can generate an array of all possible genotypes. We load in some corresponding fitness landscapes from text files.

In [2]:
function get_genotypes()
    num_genotypes = 2^num_loci
    
    bits = floor(Int, log2(num_genotypes-1))+1
    genotypes = [bin(n, bits) for n = 0:num_genotypes-1]
    
    return num_genotypes, genotypes
end

num_loci = 4
num_genotypes, genotypes = get_genotypes() # 0000 - 1111

# import the three data tables included in the supplement
landscapes = Array{DataFrame}(3)
for i = 1:3
    landscapes[i] = CSV.read(   "table$i.csv"; 
                                types=Dict(1=>String), 
                                nullable=false, 
                                rows=num_genotypes+1)
    sort!(landscapes[i], cols=1)
end
;

For each genotype, let's figure out which of the other genotypes it could mutate into (i.e. which genotypes differ from it at exactly one loci). These are its "mutational neighbors."

In [3]:
function get_mutational_neighbors()
    mutational_neighbors = Array{Int64}(num_genotypes, num_loci)
    for i = 1:num_genotypes
        for j = 1:num_loci
            neighbor = collect(genotypes[i])
            if neighbor[j] == '0'
                neighbor[j] = '1'
            else
                neighbor[j] = '0'
            end
            mutational_neighbors[i, j] = findfirst(genotypes, join(neighbor))
        end
    end
    return mutational_neighbors
end
mutational_neighbors = get_mutational_neighbors()
;

## Growth

In [4]:
function growth!(r, N)
    for i = 1:num_genotypes
        N[i] = floor(N[i] * exp(r[i]))
        if rand() < (N[i] * exp(r[i]) - floor(N[i] * exp(r[i])))
            N[i] += 1
        end
    end
end
;

## Mutation

In [5]:
function mutation!(r, N)
    oldN = copy(N)
    for i = 1:num_genotypes
        avg_num_mutants = oldN[i] * probability_mutation
        # sample from a Poisson distribution
        num_mutants = rand(Poisson(avg_num_mutants))
        for m = 1:num_mutants
            N[mutational_neighbors[i, rand(1:num_loci)]] += 1
        end
        N[i] -= num_mutants
    end
end
;

## Death by competition

In [6]:
function death!(r, N)
    ΣN = sum(N)
    for i = 1:num_genotypes
        freq = N[i] / ΣN
        N[i] = floor(freq * carrying_capacity)
        
        if rand() < (freq * carrying_capacity - floor(freq * carrying_capacity))
            N[i] += 1
        end
    end
end
;

## Running the simulation

In [7]:
# given a fitness landscape and a seed genotype, simulate population growth
# for a certain number of timesteps (by default 1200). returns a tuple of
# critical times and a trace of all population sizes at all timesteps.
function run_simulation(landscape, seed, timesteps=1200)
    if typeof(seed) == String
        seed_i = findfirst(genotypes, seed)
    else
        seed_i = seed
    end
    # starting population of all 0s, except for seed, which = carrying cap
    population=zeros(Int64, num_genotypes)
    population[seed_i] = carrying_capacity
    optimal_i = indmax(landscape)
    T_1 = 0 # time of first appearance of optimal genotype
    T_d = 0 # time to dominance
    T_f = 0 # time to fixation
    trace = Array{Int64}(timesteps,length(landscape))
    trace[1,:] = population # the first timestep is just the starting pop
    if seed_i == optimal_i
        T_1 = T_d = T_f = 1
    end
    for timestep = 2:timesteps
        # go through each stage of the simulation
        growth!(landscape, population)
        mutation!(landscape, population)
        death!(landscape, population)
        
        trace[timestep,:] = population #copy final population to the trace
        
        # check for critical times
        if T_1 == 0 && population[optimal_i] > 0
            T_1 = timestep
        end
        if T_d == 0 && population[optimal_i] > 0.5 * carrying_capacity
            T_d = timestep
        end
        if T_f == 0 && population[optimal_i] > 0.99 * carrying_capacity
            T_f = timestep
        end 
    end
    return (T_1, T_d, T_f), trace
end

# given a trace, the critical times, and an array of genotypes (by string or
# index), output a nice plot
function plot_abundance(trace, criticaltimes, genotypestoplot=1:num_genotypes)
    # if given strings, convert to indices
    if typeof(genotypestoplot) == Array{String,1}
        genotypestoplot = map(x -> findfirst(genotypes, x), genotypestoplot)
    end
    
    data = [trace[:,n] for n in genotypestoplot]
    
    #plot() only accepts a row vector of strings, hence the reshape
    labels = reshape([genotypes[n] for n in genotypestoplot], 1, :)
    
    # plot abundances
    plot(   data,
            label=labels,
            ylims=(10^0,10^10),
            yscale=:log10,
            xlims=(0,size(trace,1)),
            xlabel="Timestep",
            ylabel="Abundance of each genotype",
            legendtitle="Genotype")
    
    # add vertical lines for each of the critical times to our plot
    sub = ("1", "d", "f")
    labels = ("First appearance", "Time to dominance", "Time to fixation")
    for n=1:3
        if criticaltimes[n] != 0
            ann =  [(   criticaltimes[n],10,
                        text("<i>T</i><sub>" * sub[n] * "</sub>",10,:left))]
            plot!(  [criticaltimes[n]], 
                    label=labels[n], 
                    seriestype=:vline, 
                    linecolor=:black, 
                    primary=false, 
                    annotations=ann)
        end
    end
    plot!()
end
;

Starting with the wild-type (0000) genotype, we run a simulation on *Plasmodium vivax* treated with 53.60μM pyrimethamine:

In [8]:
landscape = copy(landscapes[3][6])
criticaltimes, trace = run_simulation(landscape, "0000")
plot_abundance(trace, criticaltimes, ["0000", "0010", "0110", "1110"])

Setting the growth rate of the first genotype along the greediest path to 0, we observe a much earlier time to dominance and time to fixation of the optimal genotype:

In [9]:
landscape[findfirst(genotypes,"0010")] = 0
criticaltimes, trace = run_simulation(landscape, "0000")
plot_abundance(trace, criticaltimes, (1, 9, 11, 15))

## Extensibility
The code can be modified to simulate more than just 4 sites of mutation:

In [15]:
num_loci = 6
num_genotypes, genotypes = get_genotypes()
mutational_neighbors = get_mutational_neighbors()

# generate a fitness landscape
landscape = rand(Normal(fit(Normal, landscapes[3][6])), num_genotypes)

criticaltimes, trace = run_simulation(landscape, "000000")

# Since there are a total of 64 genotypes in this example, attempt to choose
# the most interesting ones to plot
function interesting_genotypes()
    genotypestoplot = []
    for g = 1:num_genotypes
        if maximum(trace[:,g]) >= 100
            push!(genotypestoplot, g)
        end
    end
    return genotypestoplot
end

plot_abundance(trace, criticaltimes, interesting_genotypes())

With my naively-generated fitness landscapes, another genotype besides the optimal genotype often becomes fixed before the optimal genotype even has a chance to appear.

Multiple landscapes can be simulated and compared with ease:

In [16]:
num_loci = 4
num_genotypes, genotypes = get_genotypes()
mutational_neighbors = get_mutational_neighbors()

dosages = [ "No drug", "1.718μM", "6.389μM", "19.08μM", "53.60μM", 
            "147.4μM", "402.4μM", "1096μM", "2980μM", "8102μM"]
tic()

# run 10 simulations
times = [run_simulation(landscapes[3][l], "0000")[1] for l = 2:11]

toc() # show elapsed time

times = [time[t] for time in times, t=1:3] # fix up the array

# for output
df = DataFrame( Dosage=dosages, 
                FirstAppearance=times[:,1], 
                TimeToDominance=times[:,2], 
                TimeToFixation=times[:,3])

elapsed time: 0.422099963 seconds


,Dosage,FirstAppearance,TimeToDominance,TimeToFixation
1,No drug,1,1,1
2,1.718μM,425,1182,0
3,6.389μM,164,868,1068
4,19.08μM,174,872,1072
5,53.60μM,168,845,1045
6,147.4μM,200,610,725
7,402.4μM,133,264,309
8,1096μM,150,286,320
9,2980μM,523,589,606
10,8102μM,0,0,0
